I am getting a list of all "clean" makeup products from the categories part of the api, such that i can take their product ids and get each clean products details 

In [ ]:
import http.client
import json
import time
import pandas as pd

In [5]:
# Set up connection
conn = http.client.HTTPSConnection("sephora14.p.rapidapi.com")

# Define headers w API key
headers = {
    'x-rapidapi-key': "ec7e79be26msh4f6ce6532db872cp199109jsn37042832960c",
    'x-rapidapi-host': "sephora14.p.rapidapi.com"
}

# save the detailed product information
output_file = 'clean_makeup_product_info.json'

# Initialize
all_products_details = []
page = 1
more_products = True

while more_products:

    endpoint = f"/searchByCategory?categoryID=clean-makeup&page={page}&sortBy=NEW"
    
    try:
        # API request
        conn.request("GET", endpoint, headers=headers)
        res = conn.getresponse()
        data = res.read()
        decoded_data = data.decode("utf-8")
        
        # JSON format
        products_data = json.loads(decoded_data)
        
        if 'products' in products_data and len(products_data['products']) > 0:
            for product in products_data['products']:
                # Extract the details you want
                product_details = {
                    "brandName": product.get('brandName'),
                    "displayName": product['currentSku'].get('displayName'),
                    "heroImage": product['currentSku'].get('heroImage'),
                    "altImage": product['currentSku'].get('altImage'),
                    "onSaleData": product.get('onSaleData', 'NONE'),
                    "productId": product.get('productId'),
                    "rating": product.get('rating'),
                    "reviews": product.get('reviews'),
                    "targetUrl": product.get('targetUrl')
                }
                # add the product details to the list
                all_products_details.append(product_details)
        else:
            # stop the loop
            more_products = False
        
        # next page
        page += 1

        time.sleep(2)
    
    except Exception as e:
        print(f"An error occurred: {e}")
        more_products = False

# all product details to a file
with open(output_file, 'w') as file:
    json.dump(all_products_details, file, indent=4)

conn.close()

print(f"Collected details for {len(all_products_details)} products.")


Collected details for 376 products.


converting the json result from the api to a csv so i can get the product ids of the clean makeup products

In [7]:
# Load the JSON data
input_file = 'clean_makeup_product_info.json'
output_file = 'clean_makeup_product_info.csv'

with open(input_file, 'r') as file:
    products_data = json.load(file)

# CSV headers
csv_headers = ["brandName", "displayName", "heroImage", "altImage", "onSaleData", "productId", "rating", "reviews", "targetUrl"]

with open(output_file, 'w', newline='') as csvfile:
    csvwriter = csv.DictWriter(csvfile, fieldnames=csv_headers)

    # Write the headers to the CSV 
    csvwriter.writeheader()

    # Write each product's data to the CSV
    for product in products_data:
        csvwriter.writerow(product)

print(f"Data has been written to {output_file}")


Data has been written to clean_makeup_product_info.csv


Getting the clean makup product ids from the csv

In [8]:
clean_product_ids = df['productId'].tolist()
print(clean_product_ids)

['P513383', 'P468685', 'P512014', 'P505233', 'P512320', 'P504041', 'P481997', 'P468207', 'P505440', 'P468175', 'P507492', 'P511709', 'P481403', 'P468694', 'P468206', 'P506279', 'P513229', 'P509577', 'P512778', 'P503741', 'P504609', 'P509340', 'P468436', 'P510360', 'P509334', 'P500283', 'P455936', 'P468693', 'P448854', 'P469825', 'P507142', 'P510885', 'P509439', 'P510823', 'P510654', 'P509414', 'P448852', 'P456151', 'P500123', 'P449342', 'P509844', 'P502185', 'P507110', 'P472997', 'P455418', 'P504025', 'P506671', 'P504152', 'P511117', 'P505327', 'P505655', 'P432668', 'P510741', 'P444439', 'P506281', 'P431750', 'P509397', 'P419107', 'P502484', 'P473327', 'P510426', 'P468210', 'P504884', 'P477829', 'P441813', 'P509440', 'P448853', 'P471048', 'P505832', 'P503996', 'P507751', 'P504907', 'P466123', 'P411848', 'P468696', 'P505462', 'P471049', 'P503903', 'P509720', 'P505694', 'P503895', 'P448845', 'P504318', 'P505330', 'P457455', 'P510999', 'P505541', 'P388953', 'P504893', 'P509047', 'P481999'

In [ ]:
# Headers w API key
headers = {
    'x-rapidapi-key': "ec7e79be26msh4f6ce6532db872cp199109jsn37042832960c",
    'x-rapidapi-host': "sephora14.p.rapidapi.com"
}

# save the output
output_file = 'clean_makeup_product_details.json'

def get_product_data(clean_product_id):
    max_retries = 5  # retries
    retries = 0
    product_data = {}  # Dictionary to hold product data

    while retries < max_retries:
        try:
            # Initialize the connection inside the loop
            conn = http.client.HTTPSConnection("sephora14.p.rapidapi.com")
            
            # GET request for the current product ID
            conn.request("GET", f"/product?productID={clean_product_id}", headers=headers)
            res = conn.getresponse()
            data = res.read()

            # status code
            if res.status == 200:
                # Store the data for the current product ID
                product_data['clean_product_id'] = clean_product_id
                product_data['response'] = data.decode("utf-8")
                break 
            elif res.status == 403:
                product_data['clean_product_id'] = clean_product_id
                product_data['error'] = f"Failed to retrieve data, Status code: {res.status}"
                break 
            elif res.status == 429:
                print(f"Rate limit exceeded for product ID: {clean_product_id}. Retrying...")
                time.sleep(10) 
                retries += 1 
            else:
                product_data['clean_product_id'] = clean_product_id
                product_data['error'] = f"Failed to retrieve data, Status code: {res.status}"
                break  

            conn.close()

        except Exception as e:
            product_data['clean_product_id'] = clean_product_id
            product_data['error'] = f"An error occurred. Error: {str(e)}"
            retries += 1  

    if retries == max_retries:
        product_data['clean_product_id'] = clean_product_id
        product_data['error'] = "Max retries exceeded"

    # data to the file
    with open(output_file, 'a') as file:
        file.write(json.dumps(product_data) + "\n")

for clean_product_id in clean_product_ids:
    get_product_data(clean_product_id)
    time.sleep(2) 


In [19]:
# Initialize
all_flattened_data = []

# Open the JSON file
with open('clean_makeup_product_details.json', 'r') as file:
    for line in file:
        try:
            product = json.loads(line)
            response = json.loads(product['response'])

            # Flatten the JSON structure
            flattened_data = {
                "Product ID": product.get("clean_product_id", "N/A"),
                "Brand Name": response.get("brand", {}).get("displayName", "N/A"),
                "Product Name": response.get("displayName", "N/A"),
                "List Price": response.get("currentSku", {}).get("listPrice", "N/A"),
                "Size": response.get("currentSku", {}).get("size", "N/A"),
                "Rating": response.get("rating", "N/A"),
                "Number of Reviews": response.get("reviews", "N/A"),
                "Ingredient Description": response.get("currentSku", {}).get("ingredientDesc", "N/A"),
                "Full Product URL": response.get("fullSiteProductUrl", "N/A"),
            }

            image_urls = response.get("currentSku", {}).get("alternateImages", [])
            for idx, image in enumerate(image_urls, start=1):
                flattened_data[f"Image {idx} URL"] = image.get("image1500", "N/A")

            # add the flattened data to the list
            all_flattened_data.append(flattened_data)

        except json.JSONDecodeError as e:
            print(f"Skipping line due to JSONDecodeError: {e}")


csv_file = "clean_makeup_product_details.csv"

keys = set().union(*(d.keys() for d in all_flattened_data))

with open(csv_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=keys)
    writer.writeheader()
    writer.writerows(all_flattened_data)

print(f"Data successfully written to {csv_file}")


Data successfully written to clean_makeup_product_details.csv
